### Main tables

This script produces the seat share and competitiveness tables, for both the main text and the appendices.

Those tables are simply histograms and mean values (for the seat shares) or integrals of the competitive band (for the competitiveness tables).

The tables are constructed in LaTeX, and so most of what this script does is just 

In [1]:
import os, glob, json, re, math

import pprint

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
%matplotlib inline

pd.options.display.max_columns = 100
pd.options.display.float_format = lambda x: '%.1f' % x

import seaborn as sns
sns.set(rc={"figure.figsize": (6, 0.75)})
sns.set_style("white")

from dist_tools import *

In [2]:
pol_states = ['FL', 'IL', 'LA', 'MD', 'MN', 'NC', 'PA', 'TN', 'TX', 'VA', 'WI',
              "NC_race", "TX_senate"]

method_full = {"axis_ratio" : "Axis Ratio", "dist_a" : "Areal Distance", "dist_p" : "Population Distance",
               "dyn_radius" : "Dynamic Radius", "ehrenburg" : "Inscribed Circles", "exchange" : "Exchange", 
               "harm_radius" : "Harmonic Radius", "hull_a" : "Hull Area", "hull_p" : "Hull Population", 
               "inertia_a" : "Inertia Area", "inertia_p" : "Inertia Population", "mean_radius" : "Mean Radius", 
               "polsby" : "Isoperimeter Quotient", "power" : "Power Diagram", "reock" : "Circumscribing Circles", 
               "rohrbach" : "Distance to Perimeter", "split" : "Split-Line", "path_frac" : "Path Fraction",
               "107" : "107th Congress", "111" : "111th Congress", "114" : "114th Congress"}

In [3]:
try: # Removing dependence on database

    seats = pd.read_sql("select usps, fips, name, epsg, seats from states where fips < 57 order by usps;", con = cen_con)
    seats.to_csv("data/states.csv", index = False)
    
except: pass
    
seats = pd.read_csv("data/states.csv", index_col = "usps").seats.to_dict()
state_names = pd.read_csv("data/states.csv", index_col = "usps").name.to_dict()

## Load the data

This reads in the seat shares and vote share data from the jq'ed data files, for the 11 states with available election data.

In [4]:
rf = {s : {m : [] for m in method_full} for s in pol_states}
ds = {s : {m : {} for m in method_full} for s in pol_states}

tag = "190717"

# jdir = "/media/jsaxon/brobdingnag/data/c4_redux/{}/".format(tag)
jdir = "data/c4_redux/{}/".format(tag)

for s in pol_states:
    with open(jdir + "/{}_redux.json".format(s.lower())) as fi:
        for line in fi:
            
            if "nan" in line: continue
            
            j = json.loads(line)

            m = j["UID"].split("/")[1]

            if "TX" in s:
                if m == "axis_ratio":
                    if j["PopulationDeviation"] > 0.10: continue
                elif j["PopulationDeviation"] > 0.05: continue
            elif m == "axis_ratio":
                if j["PopulationDeviation"] > 0.05: continue
            elif j["PopulationDeviation"] > 0.02:
                
                # IL split share is a special case, 
                #   with a single map and a 2.1% deviation.
                if not ((s == "IL") and (m == "split")):
                    continue

            rf[s][m].extend(j["RepFrac"])
            
            for ey, es in j["DemSeats"].items():
                if ey not in ds[s][m]: ds[s][m][ey] = []
                ds[s][m][ey].append(es)

## Vote Share

There are just two things to do:
1. create the histogram of competitiveness
2. record the number of competitive seats, and

and then of course munging the dataframe a bit.

In [5]:
competitive_dfs = []
ST = ["IL", "MD", "NC", "TX"]

for s in ST:

    print(s, end = " ")
    competitive = []
    for m in rf[s]:
        
        if "1" in m: continue

        # Create the "mini histograms"
        plot_share(rf, s, m, seats[s], True, True)
        
        # Calculate the "mass" of competitive seats:
        #  seats * (Ncompetitive / Nsim)
        competitive.append([m, (seats[s] / len(rf[s][m])) * \
                                  len([1 for v in rf[s][m] if math.fabs(v - 0.5) < 0.025]),
                            "XZcomp_hist/{}_{}ZX".format(s.lower(), m)])
        
    # Pandas wrangling, to get the method, competitiveness and figure names
    cdf = pd.DataFrame(competitive, columns = ["Method", "# Comp.", "Figure"])
    cdf.replace({"Method" : method_full}, inplace = True)
    cdf.set_index(["Method"], inplace = True)
        
    # Stash these on a list....
    competitive_dfs.append(cdf)
    
# Concatenate them together,
cdf = pd.concat(competitive_dfs, axis = 1)

#  ... and built the multi-index as a product.
cdf.columns = pd.MultiIndex.from_product([ST, ["# Comp.", "Rep. Vote Share"]])
cdf.index.name = None
cdf.sort_index(inplace = True)

cdf

IL MD NC TX 

This simply creates the TeX formatting, directing the graphics towards the right path, etc.

In [7]:
tex = cdf.to_latex(na_rep = "", column_format = "l" + "c" * 2 * len(ST),
                                multicolumn_format = "c")

caption = """
The vote shares accruing to Republicans are plotted for
  all districts of each map, and for all available elections,
  leading to one distribution for each state and method.
The consistency in the shapes of the distributions across methods
  suggests that the many methods do not differ in their treatment of the two parties.
The different shapes for the four states shows the impact of
  political geography on partisan representation.
Republican vote shares in excess of 0.5 correspond to Republican wins;
  the integral up to 0.5 corresponds to the Democratic seat share,
  as shown for Pennsylvania in Figure~\\ref{tab:PA_seats}.
The part of the distribution close to 0.5 are competitive races.
To the left of each distribution,
  I tabulate the number of competitive races, calculated as the integral of the vote share 
  distribution between 0.475 and 0.525.
As for seat shares, the level of competitiveness is quite consistent across measures.
"""


tex = re.sub(r" *([A-Y]{2}) \& *", r" \multicolumn{2}{c}{ \selectfont \1} ", tex)

tex = tex.replace("Comp", "\\fontsize{8.5}{12}\selectfont Comp")
tex = tex.replace("Rep", "\\fontsize{8.5}{12}\selectfont Rep")

for k, v in state_names.items():
    tex = tex.replace(k, v)

tex = tex.replace("XZ", "\includegraphics[width=7em]{")
tex = tex.replace("\_", "_")
tex = tex.replace("ZX", "}")

# tex = tex.replace("nan", "")
tex = re.sub("None", "\includegraphics[width=7em]{mini_hist/blank}", tex)

tex = tex.replace("toprule", "hline \hline")
tex = tex.replace("\midrule", "\hline")
tex = tex.replace("\\bottomrule", " \hline \hline")

tex = tex + "\caption{" + caption + "}"
tex = tex + "\label{tab:competitiveness}"

tex = "\n\\begin{table}\n\\renewcommand{\\arraystretch}{1.3}\n " + tex + "\n\\end{table}\n "

tex = re.sub(r"(Split-Line)(.*)([0-9].[0-9])(.*)([0-9].[0-9])(.*)([0-9].[0-9])(.*)([0-9].[0-9])", 
             r"\\raisebox{0.7em}{\1} \2 \\raisebox{0.7em}{\3} \4 \\raisebox{0.7em}{\5} \6 \\raisebox{0.7em}{\7} \8 \\raisebox{0.5em}{\9}", tex)
# tex = "^NT:\n\n" + tex

with open("tex/competitiveness_table.tex", "w") as o:
    o.write(tex)

# Seat Shares

Start by retrieving the seat share (merging the votes) for the enacted maps.

This is quite time consuming and so I usually (and the replication uses) the cached results.

The `cdmap_seats()` function is in `dist_tools.py`.

In [8]:
if True:
    
    for ri, row in pd.read_csv("data/enacted_seat_shares.csv").iterrows():
        ds[row.State][str(row.Method)][str(row.Year)] = [row.Seats]

else:
        
    for s in pol_states:

        usps = s.split("_")[0]
        
        vote_file = None
        if "senate" in s:
            vote_file = s

        print(s, ":", end = " ")
        for ssn in [107, 111, 114]:
            print(ssn, end = " ")

            dseat_share = cdmap_seats(ssn, usps, vote_file)

            for y, dshare in dseat_share.items():

                ds[s][str(ssn)][str(y)] = [dshare * seats[usps]]

        print()
        

In [9]:
sns.set(rc={"figure.figsize": (2.5, 0.5)})
sns.set_style("white")

df_list = []

for s in pol_states:

    print(s, end = " ")
    
    years = set(sum([list(ds[s][k].keys()) for k in ds[s]], []))
    
    for y in years:
    
        min_seats = int(min(sum([ds[s][m][y] for m in method_full if y in ds[s][m]], [])))
        max_seats = int(max(sum([ds[s][m][y] for m in method_full if y in ds[s][m]], [])))
        bins = np.arange(min_seats-0.5, max_seats+0.6)

        # f, ax = plt.subplots(len(methods), sharex=True, sharey=True)
        for mi, m in enumerate(method_full):
            
            if not y in ds[s][m]: continue
                        
            f, ax = plt.subplots(1, sharex=True, sharey=True)

            dseats = sorted(ds[s][m][y])

            if len(dseats) > 1:

                # * 2 for seaborn bug: https://stackoverflow.com/questions/42404074/
                seats90 = set(dseats[round(len(dseats)*0.1):round(len(dseats)*0.9)])
                seats90 = list(np.arange(min(seats90), max(seats90)+0.1))

                ## Obnoxious seaborn/matplotlib warning aout kde keyword deprecation.  Ignore.
                with warnings.catch_warnings():
                    
                    warnings.simplefilter("ignore")

                    # This just marks the seats within the 10-90th percentiles.
                    sns.distplot(seats90 * 2, ax = ax, bins = bins, kde = False, 
                                 hist_kws = {"alpha" : 0.1, "color" : "black",
                                             "weights" : [0.5] * len(seats90) * 2})

                    sns.distplot(dseats * 2, ax = ax, bins = bins, norm_hist = True, kde = False, 
                                 hist_kws={"alpha" : 1.0, "color" : "#4DAFFF",
                                           "weights" : [0.5] * len(dseats) * 2})

            if len(dseats):
                    
                avg_seats = sum(dseats) / len(dseats)
                ax.plot([avg_seats, avg_seats], [0, 1], linewidth = 3, linestyle = "solid", 
                        c = "r" if m[0] == "1" else "k")
                
                df_list.append([s, m, y, avg_seats])

            sns.despine(left = True)
            ax.set_xlim(bins[0], bins[-1])
            ax.set_ylim(0, 1)
            ax.set_yticks([])
            ax.set_xticks(np.arange(int(min_seats), max_seats + 0.1))

            f.savefig("mini_hist/{}_{}_{}_ax.pdf".format(s, y, m), bbox_inches='tight', pad_inches=0)

            ax.set_xticks([])
            f.savefig("mini_hist/{}_{}_{}.pdf".format(s, y, m), bbox_inches='tight', pad_inches=0)
                
            plt.close('all')


FL IL LA MD MN NC PA TN TX VA WI NC_race TX_senate 

In [10]:
df = pd.DataFrame(data = df_list, columns = ["State", "Method", "Year", "Seats"])
df["Year"] = df["Year"].astype(int)
df.sort_values(by = ["State", "Method", "Year"], inplace = True)

test = df.set_index(["Method", "State", "Year"])
test["file"] = test.index.map(lambda x : "XZ{}_{}_{}ZX".format(x[1], x[2], x[0]))
test = test.unstack(level = [1, 2])
test = test.reorder_levels([1, 2, 0], axis=1)
test.index = pd.Series(test.index).map(method_full)
test.sort_index(level = [0, 1], axis=1, inplace=True)
test.sort_index(inplace = True)


caption = """\caption{{Votes from presidential elections in {} are aggregated from precinct-level returns, into maps simulated with each algorithm or compactness metric. 
             The seats expected to accrue to Democrats (mean across maps) are displayed numerically as well as by a solid black line.
             The normalized distribution of seats per metric/algorithm is shown in blue and the 10-90\% range of possible seats is highlighted in gray.
             The same re-aggregation is performed for enacted maps used for the 107th, 111th, and 114th Congresses and shown in red.
             Since reapportionment shifts the number of seats per state,
               the entries for the 107th and 111th Congresses are the Democratic share,
               times the {} assigned after the 2010 Census.
             }}"""

for s in pol_states:
    
    print(s, end = " ")

    usps = s.split("_")[0]

    years = sorted(list(df[df.State == s].Year.unique()))
    if s == "TX": years = [2004, 2008, 2012, 2016]
    if s == "TX_presidential": years = [2004, 2008, 2012, 2016]
    if s == "TX_senate": years = [2008, 2012, 2014, 2018]
    
    
    for y in years: 
        test[s][y]["file"][test[s][y]["file"].isnull()] = "XZblankZX"
    
    table = test[s][years].to_latex(column_format = "l" + " rm{7em}" * len(years))
    table = table.replace("XZ", "\includegraphics[width=7em]{mini_hist/")
    table = table.replace("\_", "_")
    table = table.replace("ZX", "}")
    
    for y in years:
        if "senate" in s:
            table = re.sub(r"Seats & *file", "\multicolumn{{2}}{{c}}{{{} US Senate}}".format(y), table, count = 1)
        else:
            table = re.sub(r"Seats & *file", "\multicolumn{{2}}{{c}}{{{} Presidential}}".format(y), table, count = 1)
    
    table = re.sub("Year.*", "", table)
    table = re.sub("Method.*", "", table)
    table = table.replace("nan", "")
    # table = re.sub("None", "", table)
    table = re.sub("None", "\includegraphics[width=7em]{mini_hist/blank}", table)
    
    table = table.replace("toprule", "hline \hline \\\\")
    table = table.replace("\midrule", "\\\\ \hline \\\\")
    table = table.replace("bottomrule", "hline \hline")
    table = table.replace("Areal Distance", "\\\\ \hline \\\\ \nAreal Distance")

    table = table.replace("split", "split_ax")
    if "race" in s: table = table.replace("power", "power_ax")
    
    table = table + caption.format(state_names[usps], seats[usps])
    table = table + "\label{{tab:{}_seats}}".format(s)
    table = "\n\\begin{table}\n\\renewcommand{\\arraystretch}{0.7}\n " + table + "\n\\end{table}\n "

    # table = "^NT\n\n" + table
    # with open("tex/{}_table".format(s), "w") as o: o.write(table)

    with open("tex/{}_table.tex".format(s), "w") as o: o.write(table)

df[df.Method.str.contains("1")].reset_index(drop = True).to_csv("data/enacted_seat_shares.csv", index = False)

FL IL LA MD MN NC PA TN TX VA WI NC_race TX_senate 

In [13]:
test["PA"]#[[2004, 2008, 2012, 2016]]

Year                    2000                           2004  \
                       Seats                     file Seats   
Method                                                        
107th Congress           8.6          XZPA_2000_107ZX   7.7   
111th Congress           8.5          XZPA_2000_111ZX   8.5   
114th Congress           6.0          XZPA_2000_114ZX   6.0   
Areal Distance           8.4       XZPA_2000_dist_aZX   7.6   
Axis Ratio               8.2   XZPA_2000_axis_ratioZX   7.6   
Circumscribing Circles   8.4        XZPA_2000_reockZX   7.6   
Distance to Perimeter    8.0     XZPA_2000_rohrbachZX   7.2   
Dynamic Radius           8.2   XZPA_2000_dyn_radiusZX   7.5   
Exchange                 8.2     XZPA_2000_exchangeZX   7.6   
Harmonic Radius          8.1  XZPA_2000_harm_radiusZX   7.2   
Hull Area                8.5       XZPA_2000_hull_aZX   7.5   
Hull Population          8.1       XZPA_2000_hull_pZX   7.3   
Inertia Area             8.3    XZPA_2000_inertia_aZX   7.5   
Inertia Population       8.1    XZPA_2000_inertia_pZX   7.3   
Inscribed Circles        8.0    XZPA_2000_ehrenburgZX   7.6   
Isoperimeter Quotient    8.3       XZPA_2000_polsbyZX   7.4   
Mean Radius              8.1  XZPA_2000_mean_radiusZX   7.5   
Path Fraction            8.0    XZPA_2000_path_fracZX   7.2   
Population Distance      8.3       XZPA_2000_dist_pZX   7.7   
Power Diagram            8.5        XZPA_2000_powerZX   7.6   
Split-Line               9.0        XZPA_2000_splitZX   7.0   

Year                                             2008  \
                                           file Seats   
Method                                                  
107th Congress                  XZPA_2004_107ZX  12.0   
111th Congress                  XZPA_2004_111ZX   8.5   
114th Congress                  XZPA_2004_114ZX  10.0   
Areal Distance               XZPA_2004_dist_aZX  11.4   
Axis Ratio               XZPA_2004_axis_ratioZX  10.6   
Circumscribing Circles        XZPA_2004_reockZX  11.0   
Distance to Perimeter      XZPA_2004_rohrbachZX  10.9   
Dynamic Radius           XZPA_2004_dyn_radiusZX  11.0   
Exchange                   XZPA_2004_exchangeZX  10.8   
Harmonic Radius         XZPA_2004_harm_radiusZX  10.4   
Hull Area                    XZPA_2004_hull_aZX  10.9   
Hull Population              XZPA_2004_hull_pZX  10.5   
Inertia Area              XZPA_2004_inertia_aZX  10.9   
Inertia Population        XZPA_2004_inertia_pZX  10.8   
Inscribed Circles         XZPA_2004_ehrenburgZX  10.2   
Isoperimeter Quotient        XZPA_2004_polsbyZX  10.7   
Mean Radius             XZPA_2004_mean_radiusZX  10.9   
Path Fraction             XZPA_2004_path_fracZX  10.5   
Population Distance          XZPA_2004_dist_pZX  10.5   
Power Diagram                 XZPA_2004_powerZX  11.4   
Split-Line                    XZPA_2004_splitZX  11.0   

Year                                             2012                           
                                           file Seats                     file  
Method                                                                          
107th Congress                  XZPA_2008_107ZX   8.6          XZPA_2012_107ZX  
111th Congress                  XZPA_2008_111ZX   8.5          XZPA_2012_111ZX  
114th Congress                  XZPA_2008_114ZX   6.0          XZPA_2012_114ZX  
Areal Distance               XZPA_2008_dist_aZX   8.5       XZPA_2012_dist_aZX  
Axis Ratio               XZPA_2008_axis_ratioZX   8.5   XZPA_2012_axis_ratioZX  
Circumscribing Circles        XZPA_2008_reockZX   8.5        XZPA_2012_reockZX  
Distance to Perimeter      XZPA_2008_rohrbachZX   8.4     XZPA_2012_rohrbachZX  
Dynamic Radius           XZPA_2008_dyn_radiusZX   8.5   XZPA_2012_dyn_radiusZX  
Exchange                   XZPA_2008_exchangeZX   8.6     XZPA_2012_exchangeZX  
Harmonic Radius         XZPA_2008_harm_radiusZX   8.3  XZPA_2012_harm_radiusZX  
Hull Area                    XZPA_2008_hull_aZX   8.5       XZPA_

In [12]:
N = []
for s in ds:
    for m in ds[s]:
        if "1" in m: continue
        if "split" in m: continue
        y = [yx for yx in ds[s][m]][0]
        N.append([s, m, len(ds[s][m][y])])
        
df = pd.DataFrame(N, columns = ["State", "Method", "N"]).set_index(["State", "Method"]).unstack(0)

df.columns = df.columns.levels[1]
df.index.name = None
df.columns.name = None
df.index = pd.Series(df.index).map(method_full)
df.sort_index(inplace = True)

df

,FL,IL,LA,MD,MN,NC,NC_race,PA,TN,TX,TX_senate,VA,WI
Areal Distance,1525,1613,1598,1589,1614,1608,730,1618,1548,1442,1442,1340,1588
Axis Ratio,34,434,1618,1610,1588,1452,351,640,1597,133,133,1511,1551
Circumscribing Circles,143,904,1389,1276,1365,1067,409,660,971,104,104,959,1238
Distance to Perimeter,462,1536,1598,1603,1532,1601,669,1592,1576,48,48,1599,1587
Dynamic Radius,1376,1609,1613,1619,1620,1618,767,1620,1619,1506,1506,1620,1614
Exchange,203,1407,1618,1620,1610,1615,657,1600,1620,740,740,1604,1619
Harmonic Radius,626,1554,1619,1620,1620,1615,1168,1601,1617,1121,1121,1619,1612
Hull Area,222,1047,1561,1581,1532,1472,711,940,1339,11,11,1084,1356
Hull Population,963,1615,1616,1605,1619,1620,763,1620,1614,1115,1115,1617,1619
Inertia Area,622,1600,1543,1599,1618,1616,761,1605,1237,1301,1301,1480,1568
